In [ ]:
import requests
import pandas as pd

json_url = 'http://localhost:1111/manutencao'
try:

    response = requests.get(json_url)
    response.raise_for_status() 
    from io import StringIO
    df = pd.read_json(StringIO(response.text))
    print("DataFrame criado com sucesso!")

except requests.exceptions.ConnectionError:
    print("ERRO: Falha na conexão. Servidor recusou a conexão (Connection Refused).")

except requests.exceptions.HTTPError as err:
    print(f"ERRO HTTP: A requisição falhou com o status: {err.response.status_code}")

except Exception as e:
    print(f"Ocorreu um erro geral: {e}")
df.head(10)

DataFrame criado com sucesso!


,id,item,tempo_manutencao,data_manutencao,setor,observacao,responsavel,tipo
0,1,Servidor Principal,90,2025-11-10,Hardware,Substituição de módulos de memória RAM.,João Victor,Preventiva
1,2,Impressora Laser B,45,2025-11-10,Escritorio,Troca de fusor e limpeza interna.,João Victor,Corretiva
2,3,Estação de Trabalho 15,20,2025-11-11,Hardware,Otimização de sistema operacional e drivers.,Anna Karolina,Preditiva
3,4,Máquina de Corte CNC,180,2025-11-12,Producao,Calibração dos eixos X e Y. Manutenção prevent...,Breno,Preditiva
4,5,Ar Condicionado Sala A,60,2025-11-12,Infraestrutura,Limpeza e recarga de gás refrigerante.,Breno,Preventiva
5,6,teste,105,2025-11-12,Almoxarifado,concerto,João Victor,Preventiva
6,7,Mesa,105,2025-12-03,RH,concerto,Alice,preventiva
7,8,Cadeira,105,2025-12-03,RH,,Alice,Preventiva
8,9,Cadeira,105,2025-12-03,RH,,João,Preventiva
9,10,Impressora Jato C,10,2025-12-08,Escritorio,Problema original: Papel atolando e ruído alto...,Anna Karolina,Preventiva


In [ ]:
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
import ipywidgets as widgets # Importação dos widgets
from IPython.display import display # Importação para exibir os widgets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from io import StringIO

# =========================================================================
# --- 1. CARREGAMENTO DOS DADOS DA API ---
# =========================================================================

json_url = 'http://localhost:1111/manutencao'
df = None

try:
    response = requests.get(json_url)
    response.raise_for_status() 
    df = pd.read_json(StringIO(response.text))
    print("DataFrame criado com sucesso!")

except requests.exceptions.ConnectionError:
    print("ERRO: Falha na conexão. Servidor recusou a conexão (Connection Refused).")
    exit()
except Exception as e:
    print(f"Ocorreu um erro geral: {e}")
    exit()

# =========================================================================
# --- 2. EXTRAÇÃO DE OPÇÕES ÚNICAS PARA INPUTS E TREINAMENTO ---
# =========================================================================

# Usamos .unique() para obter os nomes/tipos sem repetição
responsaveis_unicos = df['responsavel'].unique()
tipos_unicos = df['tipo'].unique()
setores_unicos = df['setor'].unique() # Adicionado para dropdown de Setor

print("==========================================================")
print("  Opções de Técnico(s) Histórico:", responsaveis_unicos)
print("  Opções de Tipo de Manutenção Histórico:", tipos_unicos)
print("  Opções de Setor Histórico:", setores_unicos)
print("==========================================================")

# --- 3. TREINAMENTO DO MODELO XGBOOST (O código de ML permanece o mesmo) ---

features_to_drop = ['id', 'data_manutencao', 'tempo_manutencao']
X = df.drop(features_to_drop, axis=1)
y = df['tempo_manutencao']
categorical_features = X.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

X_train, _, y_train, _ = train_test_split(X, y, test_size=0.01, random_state=42)

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42, learning_rate=0.1, max_depth=3)
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', xgb_model)])

model_pipeline.fit(X_train, y_train)
print("Modelo XGBoost treinado com sucesso.")

# =========================================================================
# --- 4. INTERFACE DE USUÁRIO E PREVISÃO COM IPYWIDGETS ---
# =========================================================================

# 4.1. CRIAÇÃO DOS WIDGETS DE INPUT
item_input = widgets.Text(description='1. Item:')
setor_input = widgets.Dropdown(options=list(setores_unicos), description='2. Setor:')
observacao_input = widgets.Textarea(description='3. Observação:')
responsavel_input = widgets.Dropdown(options=list(responsaveis_unicos), description='4. Responsável:')
tipo_input = widgets.Dropdown(options=list(tipos_unicos), description='5. Tipo:')

botao_prever = widgets.Button(description='Prever Tempo de Manutenção', button_style='success')
output_previsao = widgets.Output()

# 4.2. EXIBIÇÃO DOS WIDGETS
print("\n--- Entrada de Dados para Previsão de Tempo de Manutenção ---")
display(item_input, setor_input, observacao_input, responsavel_input, tipo_input, botao_prever, output_previsao)


# 4.3. FUNÇÃO DE PREVISÃO CHAMADA PELO BOTÃO
def prever_manutencao(b):
    with output_previsao:
        output_previsao.clear_output() # Limpa o output anterior
        
        # Cria o novo registro usando os VALORES dos widgets
        novo_registro = {
            'item': [item_input.value],
            'setor': [setor_input.value],
            'observacao': [observacao_input.value],
            'responsavel': [responsavel_input.value],
            'tipo': [tipo_input.value],
            # Coluna de data mantida para consistência, mas é descartada
            'data_manutencao': ['2026-01-15'] 
        }

        novo_dado_df = pd.DataFrame(novo_registro)
        novo_dado_df_previsao = novo_dado_df.drop('data_manutencao', axis=1, errors='ignore')

        # Faz a previsão usando o pipeline treinado
        tempo_previsto = model_pipeline.predict(novo_dado_df_previsao)
        tempo_estimado = tempo_previsto[0]

        # --- 5. RESULTADO DA PREVISÃO ---
        print("\n----------------------------------------------------------")
        print(f"Item: {novo_registro['item'][0]}")
        print(f"Setor: {novo_registro['setor'][0]}")
        print(f"Responsável: {novo_registro['responsavel'][0]}")
        print(f"Tipo: {novo_registro['tipo'][0]}")
        print(f"\nO Tempo de Manutenção ESTIMADO é de: {tempo_estimado:.2f} minutos.")
        print(f"Isso equivale a aproximadamente: {tempo_estimado / 60:.2f} horas.")
        print("----------------------------------------------------------")


# 4.4. ATRIBUIÇÃO DA FUNÇÃO AO BOTÃO
botao_prever.on_click(prever_manutencao)

# NOTA: Este código DEVE ser rodado em um ambiente que suporta widgets (Jupyter Notebook, Voila, etc.).

DataFrame criado com sucesso!
  Opções de Técnico(s) Histórico: ['João Victor' 'Anna Karolina' 'Breno' 'Alice']
  Opções de Tipo de Manutenção Histórico: ['Preventiva' 'Corretiva' 'Preditiva' 'preventiva']
  Opções de Setor Histórico: ['Hardware' 'Escritorio' 'Producao' 'Infraestrutura' 'Almoxarifado' 'RH']
Modelo XGBoost treinado com sucesso.

--- Entrada de Dados para Previsão de Tempo de Manutenção ---


Text(value='', description='1. Item:')

Dropdown(description='2. Setor:', options=('Hardware', 'Escritorio', 'Producao', 'Infraestrutura', 'Almoxarifa…

Textarea(value='', description='3. Observação:')

Dropdown(description='4. Responsável:', options=('João Victor', 'Anna Karolina', 'Breno', 'Alice'), value='Joã…

Dropdown(description='5. Tipo:', options=('Preventiva', 'Corretiva', 'Preditiva', 'preventiva'), value='Preven…

Button(button_style='success', description='Prever Tempo de Manutenção', style=ButtonStyle())

Output()